In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Load Network 15AB

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_1/models/B5R2_block5_15AB_1FC_3.h5' #model
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image/'#แก้
dataframe = pd.read_csv( '/home/yupaporn/CCA_Project/Dataset_CCA_case_Testset1312.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
validation_dir = os.path.join(base_dir, 'test')

#Train
train_df = pd.read_csv( '/home/yupaporn/CCA_Project/Dataset_CCA_case_Trainset5257.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
base_dir0 = '/media/tohn/SSD/Images/Image/'#แก้
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/CCA_Project/Dataset_CCA_case_Testset1312.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1312, 24)
(1312, 24)
Normal:  (857, 24)
Abnormal:  (455, 24)


,Unnamed: 0,Case,Abs Position,Sub Position,Class,Path Full,Path Crop,Views,fold,15AB,...,height,Rleft,Rtop,Rwidth,Rheight,Sub_class_New,tag_AjNit,tag_AjWan,Test150,Spilt
0,0,AB01_40,P1,P1,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,2,MildFattyLiver,...,311.0,0.092664,0.148873,0.513514,0.346614,AB01,NaN,NaN,False,Test
1,1,AB01_40,P2,P2,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,2,MildFattyLiver,...,385.0,0.171698,0.154849,0.560377,0.428287,AB01,NaN,NaN,False,Test
2,2,AB01_40,P4,P41,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,2,MildFattyLiver,...,640.0,0.140316,0.150865,0.667984,0.711155,AB01,NaN,NaN,False,Test
3,3,AB01_40,P5,P51,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,2,MildFattyLiver,...,587.0,0.063462,0.107041,0.690385,0.653386,AB01,NaN,NaN,False,Test
4,4,AB01_40,P3,P31,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,2,MildFattyLiver,...,562.0,0.216374,0.146881,0.672515,0.625498,AB01,NaN,NaN,False,Test


In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class_New',# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1312 validated image filenames belonging to 15 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.10783226,  0.31214914,  0.0173418 , ...,  0.4004598 ,
         0.14019933,  0.37498158],
       [ 0.41058418,  0.66091764, -0.13656126, ...,  0.23649257,
         0.63316631,  0.09244028],
       [-0.0175069 ,  0.34510681,  0.22585739, ...,  0.24183793,
         0.39001301, -0.1302003 ],
       ...,
       [ 0.31147826,  0.80987614, -0.18276736, ...,  1.19653344,
         0.1428871 ,  0.7533325 ],
       [ 0.27330738, -0.02279092,  0.0221122 , ...,  0.21970937,
        -0.15190673,  0.27240428],
       [ 0.18239234,  0.16289063, -0.16271318, ...,  0.35533604,
        -0.00221718,  0.65974718]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.107832,0.312149,0.017342,-0.183515,0.575329,-0.173988,-0.089337,-0.063323,-0.030137,0.059896,...,-0.192440,-0.159383,0.653819,-0.121369,0.138476,-0.108643,0.148073,0.400460,0.140199,0.374982
1,0.410584,0.660918,-0.136561,-0.185142,0.411921,-0.167172,0.004929,0.000339,-0.130162,-0.110030,...,-0.145702,-0.168996,0.213558,-0.044708,0.396022,-0.173823,-0.012629,0.236493,0.633166,0.092440
2,-0.017507,0.345107,0.225857,-0.165460,0.779620,-0.193235,-0.032826,-0.147712,-0.096545,0.065196,...,-0.125954,-0.136239,0.735194,-0.071153,0.282351,-0.138691,-0.005356,0.241838,0.390013,-0.130200
3,-0.020903,0.117676,-0.082830,-0.135977,0.282654,-0.220009,0.111935,-0.094879,-0.103360,0.036063,...,-0.179393,-0.162493,0.375170,-0.082911,0.298246,-0.055976,0.022169,0.057338,0.233831,-0.014273
4,0.064264,0.339072,0.294476,-0.157963,0.194470,-0.202357,-0.088955,-0.190239,-0.139357,-0.224388,...,-0.098920,-0.025339,0.175932,-0.027771,0.215618,0.174927,-0.076771,0.139155,0.083005,-0.109640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,0.744561,1.183853,-0.053521,-0.201384,0.486397,-0.161738,0.369919,0.262434,0.401863,0.046597,...,0.556524,-0.044269,0.239366,-0.006773,-0.007029,-0.143083,0.694541,0.309271,0.582698,0.681158
1308,0.002655,-0.022164,-0.106891,-0.050455,0.119427,0.226081,-0.119361,0.402494,0.157568,0.171320,...,0.420776,-0.108082,0.338135,0.070919,-0.033625,0.049810,0.067249,0.423210,-0.147668,0.224348
1309,0.311478,0.809876,-0.182767,-0.104543,0.685962,0.047534,0.061282,0.351369,0.239476,0.172857,...,-0.129816,-0.179644,0.494608,0.241756,0.758375,0.121147,0.329733,1.196533,0.142887,0.753332
1310,0.273307,-0.022791,0.022112,-0.126591,-0.079219,-0.175265,0.018624,-0.036007,0.046273,-0.022751,...,-0.036539,-0.101429,-0.133907,0.142786,0.102545,-0.169974,0.069231,0.219709,-0.151907,0.272404


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class_New'].tolist()
e= dataframe['Case'].tolist()
dataset['Case'] = e
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class_New'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,Case,Class,SubPosition,Views,Sub_class_New
0,-0.107832,0.312149,0.017342,-0.183515,0.575329,-0.173988,-0.089337,-0.063323,-0.030137,0.059896,...,-0.108643,0.148073,0.400460,0.140199,0.374982,AB01_40,Abnormal,P1,FP-A,AB01
1,0.410584,0.660918,-0.136561,-0.185142,0.411921,-0.167172,0.004929,0.000339,-0.130162,-0.110030,...,-0.173823,-0.012629,0.236493,0.633166,0.092440,AB01_40,Abnormal,P2,FP-A,AB01
2,-0.017507,0.345107,0.225857,-0.165460,0.779620,-0.193235,-0.032826,-0.147712,-0.096545,0.065196,...,-0.138691,-0.005356,0.241838,0.390013,-0.130200,AB01_40,Abnormal,P41,FP-B,AB01
3,-0.020903,0.117676,-0.082830,-0.135977,0.282654,-0.220009,0.111935,-0.094879,-0.103360,0.036063,...,-0.055976,0.022169,0.057338,0.233831,-0.014273,AB01_40,Abnormal,P51,FP-C,AB01
4,0.064264,0.339072,0.294476,-0.157963,0.194470,-0.202357,-0.088955,-0.190239,-0.139357,-0.224388,...,0.174927,-0.076771,0.139155,0.083005,-0.109640,AB01_40,Abnormal,P31,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,0.744561,1.183853,-0.053521,-0.201384,0.486397,-0.161738,0.369919,0.262434,0.401863,0.046597,...,-0.143083,0.694541,0.309271,0.582698,0.681158,350,Normal,P32,FP-B,Normal
1308,0.002655,-0.022164,-0.106891,-0.050455,0.119427,0.226081,-0.119361,0.402494,0.157568,0.171320,...,0.049810,0.067249,0.423210,-0.147668,0.224348,350,Normal,P42,FP-B,Normal
1309,0.311478,0.809876,-0.182767,-0.104543,0.685962,0.047534,0.061282,0.351369,0.239476,0.172857,...,0.121147,0.329733,1.196533,0.142887,0.753332,350,Normal,P52,FP-C,Normal
1310,0.273307,-0.022791,0.022112,-0.126591,-0.079219,-0.175265,0.018624,-0.036007,0.046273,-0.022751,...,-0.169974,0.069231,0.219709,-0.151907,0.272404,350,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/CCA_Project')
testdf = dataset
# testdf.to_csv('FeatureVector_CCA_case_Testset1312.csv') 